# Welcome to the ProgPy Ensemble Model Example!

This notebook demonstrates the use of the Ensemble Model feature, an approach that allows for the simultaneous simulation of multiple models and the aggregation of their predictions into a single forecast. This technique can improve the accuracy of predictions when you have multiple models that each represent a part of the behavior or represent a distribution of different behaviors.

The notebook consists of __two__ parts:

1. The first part involves setting up and simulating four different equivalent circuit models, each with different configuration parameters. The results of these models are then aggregated into a single ensemble model and simulated. The results are plotted for comparison. This part of the notebook also demonstrates how to handle outlier models that can skew the aggregated results.

2. The second part of the notebook demonstrates the creation of an ensemble model consisting of two different types of models: the equivalent circuit model and the electro-chemistry model. The ensemble model is simulated over time and the results are plotted.

This notebook uses battery data to illustrate the implementation and effectiveness of the Ensemble Model approach.

### Importing Modules

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from progpy import EnsembleModel
from progpy.datasets import nasa_battery
from progpy.models import BatteryElectroChemEOD, BatteryCircuit

## Part 1. Different Model Configurations

First, we will download the required battery data. We then extract the necessary features, such as current and relative time, from the downloaded data.

In [ ]:
# Download data
print('downloading data (this may take a while)...')
data = nasa_battery.load_data(8)[1]

# Prepare data
RUN_ID = 0
test_input = [{'i': i} for i in data[RUN_ID]['current']]
test_time = data[RUN_ID]['relativeTime']

Afterwards, we'll set up different versions of the circuit model with various parameters. This is done to simulate uncertainty on the parameters of the model. We create three different models with varying parameters and then combine them into an ensemble model. 


In [ ]:
print('Setting up models...')
m_circuit = BatteryCircuit(process_noise = 0, measurement_noise = 0)
m_circuit_2 = BatteryCircuit(process_noise = 0, measurement_noise = 0, qMax = 7860)
m_circuit_3 = BatteryCircuit(process_noise = 0, measurement_noise = 0, qMax = 6700, Rs = 0.055)
m_ensemble = EnsembleModel((m_circuit, m_circuit_2, m_circuit_3), process_noise = 0, measurement_noise = 0)


Next, we evaluate the models. We define a function to provide future loadings based on the test data and simulate the ensemble model to the end of the test period.


In [ ]:
# Evaluate models
print('Evaluating models...')
def future_loading(t, x=None):
    for i, mission_time in enumerate(test_time):
        if mission_time > t:
            return m_circuit.InputContainer(test_input[i])
results_ensemble = m_ensemble.simulate_to(test_time.iloc[-1], future_loading)

Let's visualize the results of the ensemble model and compare it with the ground truth from the test data. We plot the voltage over time for both the ensemble model and the actual data.

In [ ]:
plt.plot(test_time, data[RUN_ID]['voltage'], color='green', label='ground truth')
plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='red', label='ensemble')
plt.legend()
plt.show()

Note: This is a very poor performing model, since there was an outlier model (m_circuit_3), which effected the quality of the model prediction. This can be resolved by using a different `aggregation_method`. For example, median. In a real scenario, you would likely remove this model, this is just to illustrate outlier elimination.

In [ ]:
print('Updating with Median ')
m_ensemble.parameters['aggregation_method'] = np.median
results_ensemble = m_ensemble.simulate_to(test_time.iloc[-1], future_loading)
plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='orange', label='ensemble - median')
plt.legend()

## Part 2: Different Models

In the previous section, we used ensemble modeling to deal with uncertainties in model parameters. In this section, we will extend this concept to different models that have different states. Specifically, we will create an ensemble model using the equivalent circuit and electro-chemical models for a battery. These two models share one state, but they also have unique states.

By incorporating models that capture different aspects of the system into an ensemble, we aim to make more comprehensive and reliable predictions.

#### Let's setup the models.

In [ ]:
# Setup the ElectroChem model
m_electro = BatteryElectroChemEOD(process_noise = 0, measurement_noise = 0)

# Setup the Ensemble Model
m_ensemble = EnsembleModel((m_circuit, m_electro), process_noise = 0, measurement_noise=0)

#### Evaluating Models

In [ ]:
# Evaluate the ensemble model
results_ensemble = m_ensemble.simulate_to(test_time.iloc[-1], future_loading)

# Evaluate the circuit model
results_circuit1 = m_circuit.simulate_to(test_time.iloc[-1], future_loading)

# Evaluate the ElectroChem model
results_electro = m_electro.simulate_to(test_time.iloc[-1], future_loading)

#### Plotting Results

In [ ]:
plt.figure()
plt.plot(test_time, data[RUN_ID]['voltage'], color='green', label='ground truth')
plt.plot(results_circuit1.times, [z['v'] for z in results_circuit1.outputs], color='blue', label='circuit')
plt.plot(results_electro.times, [z['v'] for z in results_electro.outputs], color='red', label='electro chemistry')
plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='yellow', label='ensemble')
plt.legend()
plt.show()

From the plot, we can see the predictions made by the individual models and the ensemble model compared to the ground truth. Note that the ensemble model's result may not be exactly between the two individual models' results. This discrepancy is due to the two-step aggregation process: state transition and output calculation.

Overall, the ensemble model provides a powerful tool for capturing the various behaviors of complex systems. By effectively combining the predictive strengths of multiple individual models, we can make more reliable and robust predictions.

## Conclusion

In this notebook, we demonstrated how to implement the Ensemble Model approach using ProgPy. We learned how to create ensemble models with different configurations, and how to handle outlier models in an ensemble. We also saw how to create ensemble models with different types of models that have different states. 

The Ensemble Model approach is beneficial in scenarios where multiple models each represent a part of the behavior or represent a distribution of different behaviors. Using this approach, we can aggregate the predictions of these models into a single, potentially more accurate, forecast.

This notebook serves as a practical guide for implementing the Ensemble Model approach and should help you to use this approach effectively in your own projects.

For more ProgPy information, please refer to our ProgPy [Documentation](https://nasa.github.io/progpy/).